In [1]:
import os
dst_path = '/p/openvocabdustr/probing_midlevel_vision/code/probing-mid-level-vision'
# Change to the destination directory
os.chdir(dst_path)

from functools import partial
import torch
import timm
from models.util import load_checkpoint, initialize_backbone, prepare_state_dict

/p/openvocabdustr/probing_midlevel_vision/env/probing3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/p/openvocabdustr/probing_midlevel_vision/env/probing3d/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025847094/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/p/openvocabdustr/probing_midlevel_vision/env/probing3d/lib/python3.9/site-packages/m

In [2]:
GLOBAL_POOL = False

checkpoints = {
    "vitb16": {
        "url": "https://dl.fbaipublicfiles.com/mae/pretrain/mae_pretrain_vit_base.pth",
        "filename": "mae_vitb16.pth",
    },
}


class MAE_ViT(timm.models.vision_transformer.VisionTransformer):
    """ Vision Transformer with support for global average pooling
    Copied from: https://github.com/facebookresearch/mae/blob/main/models_vit.py
    """
    def __init__(self, **kwargs):
        super(MAE_ViT, self).__init__(
            patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4,
            qkv_bias=True, norm_layer=partial(torch.nn.LayerNorm, eps=1e-6, **kwargs))

        self.global_pool = GLOBAL_POOL
        if self.global_pool:
            norm_layer = partial(torch.nn.LayerNorm, eps=1e-6, **kwargs)
            embed_dim = 768
            self.fc_norm = norm_layer(embed_dim)

            del self.norm  # remove the original norm

        self.head = torch.nn.Identity()

    def forward_features(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks:
            x = blk(x)

        if self.global_pool:
            x = x[:, 1:, :].mean(dim=1)  # global pool without cls token
            outcome = self.fc_norm(x)
        else:
            x = self.norm(x)
            outcome = x[:, 0]


        return outcome


def load_model(arch: str, **kwargs):
    assert arch == "vitb16"
    model = MAE_ViT()
    ckpt = load_checkpoint(**checkpoints[arch])["model"]
    if GLOBAL_POOL:
        ckpt["fc_norm.weight"] = ckpt["norm.weight"]
        ckpt["fc_norm.bias"] = ckpt["norm.bias"]
        del ckpt["norm.weight"]
        del ckpt["norm.bias"]

    model.load_state_dict(ckpt)
    return model

In [3]:
for arch in checkpoints.keys():
    model = load_model(arch=arch)
    print(model)
    out = model(torch.randn(1, 3, 224, 224))
    print(out.shape)

MAE_ViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        